# Hyperparameter Search, Cross Validation, and Metrics Exercise

Return to your logistic regression exercises and expand your answer by doing the following:

1. Select at least 3 hyperparameters from the Logistic Regression model (see documentation [https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
    * Select at least 2 values for each of your selected hyperparameters.
2. Select at least 3 metrics to track.
3. Perform a Grid Search search using KFold cross validation with k=5
4. Examine the results...
    * Is the most accurate model the one with the best values in your other metrics?
    * Can you find any evidence of a "tradeoff" where one metric is typically high when another is low?
    * Is there anything you find particularly interesting about the results?

In [2]:
# Lets do grid search cross validation on a Decision Tree:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Load the data
fish_dataset = pd.read_csv('../datasets/fish/Fish.csv')

# Note, we're not doing a test train split because the cross validation built into 
# Grid and Random search perform this splitting for each CV fold.
# But we still need to split the labels and features
labels = fish_dataset['Species']
features = fish_dataset.drop(columns=['Species'])

features.head()

,Weight,Length1,Length2,Length3,Height,Width
0,242.0,23.2,25.4,30.0,11.5200,4.0200
1,290.0,24.0,26.3,31.2,12.4800,4.3056
2,340.0,23.9,26.5,31.1,12.3778,4.6961
3,363.0,26.3,29.0,33.5,12.7300,4.4555
4,430.0,26.5,29.0,34.0,12.4440,5.1340


In [11]:
# 1. Selecting params and values
params = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [1.0, 2.0, 5.0],
    'fit_intercept': [True, False],
    'multi_class': ['ovr', 'multinomial'],
    'max_iter': [10000] # Specifying just one large value to give our models a good chance to converge
}

# 2. Selecting metrics
# Because this is multi-class we need to use one of the modified precision/recall values.
# The default only works for binary classification.
metrics = ['accuracy', 'precision_weighted', 'recall_weighted']

# 3. Perform Grid Search
## 3a you first make a single instance of the class for our model
model_base = LogisticRegression()

## 3b you then specify the model, params, and any scoring metrics you wish to track.
##    When specifying multiple metrics you must use the refit param as well to indicate which of these is
##    your "main" metric that will be considered "best" when you ask for the best_estimator_ or best_params_ 
##    later on.

## Also note: error_score=np.nan. Some combinations of hyperparemters are illegal. If such an error occurs
##            this value will be used as the "score" for all metrics for that combo. We choose nan to clearly
##            indicate it was a failed run.

## The default cross validator is kfold with k==5 so you don't have to explicitly specify it
gird_searcher = GridSearchCV(model_base, params, scoring=metrics, refit='accuracy', error_score=np.nan) 

## 3c then you fit the model
gird_searcher.fit(features, labels) 
print('done') # Just so we know when its finished.

## Note: Look at all those errors and warnings :(

/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

 

/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-package

/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver

done


/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tylerbettilyon/opt/anaconda3/envs/sb-data-science/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
   

In [27]:
print("Best parameters set found on development set:")
print()
print(gird_searcher.best_params_, gird_searcher.best_score_)
print()
print("Grid scores on development set:")
print()

available_metrics = gird_searcher.cv_results_.keys()

# Take a look at all the data we have access to... wow!
for v in available_metrics:
    print(v)
    
# Each of these corresponds to an array or 2d array where each row represents one
# of our trained models...
# Note that evidently MANY of our desired combinations were erronions, since there are lots of nans
# in the accuracy department.
print()
print(gird_searcher.cv_results_['mean_test_accuracy'])

Best parameters set found on development set:

{'C': 5.0, 'fit_intercept': False, 'max_iter': 10000, 'multi_class': 'ovr', 'penalty': 'l2'} 0.930241935483871

Grid scores on development set:

mean_fit_time
std_fit_time
mean_score_time
std_score_time
param_C
param_fit_intercept
param_max_iter
param_multi_class
param_penalty
params
split0_test_accuracy
split1_test_accuracy
split2_test_accuracy
split3_test_accuracy
split4_test_accuracy
mean_test_accuracy
std_test_accuracy
rank_test_accuracy
split0_test_precision_weighted
split1_test_precision_weighted
split2_test_precision_weighted
split3_test_precision_weighted
split4_test_precision_weighted
mean_test_precision_weighted
std_test_precision_weighted
rank_test_precision_weighted
split0_test_recall_weighted
split1_test_recall_weighted
split2_test_recall_weighted
split3_test_recall_weighted
split4_test_recall_weighted
mean_test_recall_weighted
std_test_recall_weighted
rank_test_recall_weighted
[       nan 0.89274194        nan        nan 0.86

In [32]:
# Lets use pandas to explore this data in a nicer way.
results_df = pd.DataFrame({
    'mean_accuracy': gird_searcher.cv_results_['mean_test_accuracy'],
    'mean_precision': gird_searcher.cv_results_['mean_test_precision_weighted'],
    'mean_recall': gird_searcher.cv_results_['mean_test_recall_weighted'],
})

params_df = pd.DataFrame(gird_searcher.cv_results_['params'], columns=gird_searcher.cv_results_['params'][0].keys())

all_results = pd.concat([results_df, params_df], axis=1).sort_values('mean_accuracy', ascending=False)

# Lets filter out the ones with nans, then list them!
# (36 is the total number of models, so I know 36 is enough to display them all.)
all_results.dropna().head(36)

,mean_accuracy,mean_precision,mean_recall,C,fit_intercept,max_iter,multi_class,penalty
31,0.930242,0.913372,0.930242,5.0,False,10000,ovr,l2
19,0.923790,0.902442,0.923790,2.0,False,10000,ovr,l2
34,0.923589,0.928976,0.923589,5.0,False,10000,multinomial,l2
7,0.917339,0.885148,0.917339,1.0,False,10000,ovr,l2
22,0.911089,0.906474,0.911089,2.0,False,10000,multinomial,l2
10,0.904839,0.902308,0.904839,1.0,False,10000,multinomial,l2
13,0.899194,0.899665,0.899194,2.0,True,10000,ovr,l2
25,0.892944,0.901287,0.892944,5.0,True,10000,ovr,l2
1,0.892742,0.876169,0.892742,1.0,True,10000,ovr,l2
28,0.879435,0.897725,0.879435,5.0,True,10000,multinomial,l2


## Note:

Our precision and recall scores are always quite close to our overall accuracy, which is good!

### Challenge yourself...:

Repeat this process, but with Random Search and ask yourself the following:

* Was it faster?
* Was the "best" model similar in terms of the performance metrics you tracked?